<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.


In [ ]:
import pandas as pd

import re
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

## Подготовка

In [ ]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    print('Не удалось прочитать файл')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
data = data.drop('Unnamed: 0', axis =1)

In [ ]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
data.duplicated().sum()

0

In [ ]:
data.groupby(by='toxic').count()

,text
toxic,
0,143106
1,16186


In [ ]:
#m = Mystem()
nltk.download('stopwords')
stopwords_list = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def get_wordnet_pos(word):
    #"""Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:


def lemmatize_text(text):
    text = text.lower()
    #lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', text)
    #split_text = re.split(' ', text)
    word_list = nltk.word_tokenize(cleared_text)
    word_list = [word for word in word_list if word not in stopwords_list]
    word_list = [lemmatizer.lemmatize(word,  get_wordnet_pos(word)) for word in word_list]
    lemma_text = " ".join(word_list)
    return lemma_text


In [ ]:

data['lemma_text'] = data['text'].progress_apply(lemmatize_text)
data.head(10)

100%|██████████| 159292/159292 [12:22<00:00, 214.64it/s]


,text,toxic,lemma_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page
5,"""\n\nCongratulations from me as well, use the ...",0,congratulation well use tool well talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker piss around work
7,Your vandalism to the Matt Shirvington article...,0,vandalism matt shirvington article revert plea...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry word nonsense offensive anyway intend wr...
9,alignment on this subject and which are contra...,0,alignment subject contrary dulithgow


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        159292 non-null  object
 1   toxic       159292 non-null  int64 
 2   lemma_text  159292 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


In [ ]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

8.841344371679229

In [ ]:
dict_classes={0:1, 1:class_ratio}
dict_classes

{0: 1, 1: 8.841344371679229}

**Вывод:** проведена лемматизация текстовых данных. При ознакомлении с данными выявлен дисбаланс классов (нетоксичных комментариев оказалось гораздо больше, чем токсичных в 8,8 раза). Для увеличения точности работы моделей необходимо учитывать это при делении выборок и обучении моделей

## Обучение

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=613, stratify=data['toxic'])

In [ ]:
count_tf_idf = TfidfVectorizer()

In [ ]:
features_train = count_tf_idf.fit_transform(train['lemma_text'])

In [ ]:
features_test = count_tf_idf.transform(test['lemma_text'])

In [ ]:
target_train = train['toxic']
target_test = test['toxic']

Подготовим таблицу для сбора результатов

In [ ]:
result=pd.DataFrame({'model': [], 'f1':[]})
result

,model,f1


In [ ]:
def append_result(model, result, f_train, t_train):
    scores = cross_val_score(model, f_train, t_train, scoring='f1', cv=3)
    f1 = scores.mean()
    result = result.append({'model': model, 'f1':f1}, ignore_index=True)
    return result

In [ ]:
model = LogisticRegression(class_weight=dict_classes)
model.fit(features_train, target_train)
result = append_result(model, result,features_train, target_train)
result

,model,f1
0,"LogisticRegression(class_weight={0: 1, 1: 8.84...",0.756488


In [ ]:
param = {
    'max_depth': [n for n in range(1, 25, 2)],
    'n_estimators':[i for i in range(1, 25,2)],
            }
grid_random_forest =  GridSearchCV(RandomForestClassifier(random_state=613, class_weight=dict_classes), param, scoring='f1', n_jobs=-1, cv=3);
grid_random_forest.fit(features_train, target_train)
grid_random_forest.best_params_

{'max_depth': 23, 'n_estimators': 23}

In [ ]:
rf_model = RandomForestClassifier(random_state=613, class_weight=dict_classes, max_depth = grid_random_forest.best_params_['max_depth'], n_estimators=grid_random_forest.best_params_['n_estimators'])
rf_model.fit(features_train, target_train)
result = append_result(rf_model, result, features_train, target_train)
result

,model,f1
0,"LogisticRegression(class_weight={0: 1, 1: 8.84...",0.756488
1,"(DecisionTreeClassifier(max_depth=23, max_feat...",0.375118


In [ ]:
cat_model = CatBoostClassifier(verbose=3, iterations=50)
cat_model.fit(features_train, target_train)
result = append_result(cat_model, result, features_train, target_train)
result

Learning rate set to 0.5
0:	learn: 0.3334058	total: 2.66s	remaining: 2m 10s
3:	learn: 0.2122789	total: 8.75s	remaining: 1m 40s
6:	learn: 0.1944144	total: 14.6s	remaining: 1m 29s
9:	learn: 0.1826768	total: 20.4s	remaining: 1m 21s
12:	learn: 0.1738423	total: 26s	remaining: 1m 13s
15:	learn: 0.1669901	total: 31.6s	remaining: 1m 7s
18:	learn: 0.1612437	total: 37.3s	remaining: 1m
21:	learn: 0.1573258	total: 42.9s	remaining: 54.6s
24:	learn: 0.1540534	total: 48.5s	remaining: 48.5s
27:	learn: 0.1507028	total: 53.9s	remaining: 42.4s
30:	learn: 0.1477089	total: 59.4s	remaining: 36.4s
33:	learn: 0.1441658	total: 1m 4s	remaining: 30.6s
36:	learn: 0.1418748	total: 1m 10s	remaining: 24.8s
39:	learn: 0.1397199	total: 1m 15s	remaining: 19s
42:	learn: 0.1375439	total: 1m 21s	remaining: 13.3s
45:	learn: 0.1357219	total: 1m 26s	remaining: 7.55s
48:	learn: 0.1341628	total: 1m 32s	remaining: 1.88s
49:	learn: 0.1336514	total: 1m 34s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3443817	total: 1.85s	r

,model,f1
0,"LogisticRegression(class_weight={0: 1, 1: 8.84...",0.756488
1,"(DecisionTreeClassifier(max_depth=23, max_feat...",0.375118
2,<catboost.core.CatBoostClassifier object at 0x...,0.721241


In [ ]:
sgd_model = SGDClassifier()
hyperparams = [{'loss':['hinge', 'log', 'modified_huber'],
                'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'],
                'eta0':[0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
                'random_state':[613],
                'class_weight':[dict_classes]}]
grid_sgd = GridSearchCV(sgd_model, hyperparams, scoring='f1',cv=3)
grid_sgd.fit(features_train, target_train)
SGDC_best_params = grid_sgd.best_params_
print(SGDC_best_params)

{'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 613}


In [ ]:
sgd_model = SGDClassifier()
sgd_model.set_params(**SGDC_best_params)
sgd_model.fit(features_train, target_train)
result = append_result(sgd_model, result, features_train, target_train)
result

,model,f1
0,"LogisticRegression(class_weight={0: 1, 1: 8.84...",0.756488
1,"(DecisionTreeClassifier(max_depth=23, max_feat...",0.375118
2,<catboost.core.CatBoostClassifier object at 0x...,0.721241
3,"SGDClassifier(class_weight={0: 1, 1: 8.8413443...",0.759261


In [ ]:
best_model = result.sort_values(by='f1', ascending=False).iloc[0,0]
best_rmse = result.sort_values(by='f1', ascending=False).iloc[0,1]
print('Модель:', best_model)
print('f1', best_rmse)

Модель: SGDClassifier(class_weight={0: 1, 1: 8.841344371679229}, eta0=0.01,
              learning_rate='constant', loss='modified_huber',
              random_state=613)
f1 0.7592613478071328


**ВЫБОР МОДЕЛИ:** Лучшую метрику показал SGD-классификатора с учетом баланса классов, так же по критерию заказчика прошла логистическая регрессия с балансом классов, но ее метрика оказалась немного ниже.

In [ ]:
y_pred = best_model.predict(features_test)
best_f1_test = f1_score(target_test, y_pred)
print('f1 ', best_f1_test)

f1  0.7583874458874459


In [ ]:
if best_f1_test > 0.75:
    print('Модель соответствует требованиям заказчика')
else:
    print('Модель НЕ соответствует требованиям заказчика')

Модель соответствует требованиям заказчика


**Вывод:** Наилучший результат показала модель SGD-классификатора с учетом баланса классов. Модель показала метрику F1 почти 0,76 как на кросс-валидации так и на тестовых данных

## Выводы

К анализу были предоставлены размеченные сведения полученных Интернет-магазином «Викишоп» правок к описаниям и комментариев к товарам с подготовленной разметкой о токсичности.
В ходе работы над проектом:
- исключены незначимые символы из текста комментариев
- проведена лемматизация комментариев
- выявлен дисбаланс классов, который был учтен при подготовке данных и обучении моделей
- обучена модель TF-IDF для генерации из текста комментариев числовых признаков для обучения модели
- обучены модели:

    1) Логистическая регрессия
    
    2) Случайный лес
    
    3) CatBoost-классификатор
    
    4) SGD-классификатор
    
Наилучший результат показала модель  модель SGD-классификатора с учетом баланса классов. Модель показала метрику F1 = 0,76 как на кросс-валидации так и на тестовых данных